<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Data Analysis</center></h1>
<h2><center>Seminar: Boosting </center></h2>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

# Load Dataset

Load data from `data/x_data.tsv` and `data/y_data.tsv`. 

These are descriptive features and target feature. Put them in `X` and `y` respectively.

In [ ]:
df_X = pd.read_csv('./data/x_train.csv', sep=';')
df_y = pd.read_csv('./data/y_train.csv', sep=';', header=None,
                  names=['target'])

In [ ]:
df_X.head()
df_y.head()

In [ ]:
df_X.shape

In [ ]:
df_y.shape

In [ ]:
X = df_X.values
y = df_y.values

# Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=123, test_size=0.2)

In [ ]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), 
                         n_estimators=1000, 
                         learning_rate=0.01, 
                         random_state=123)

In [ ]:
ada.fit(X_train, y_train)

In [ ]:
scores_train = []
scores_valid = []

for y_pred in ada.staged_predict_proba(X_train):
    scores_train.append(roc_auc_score(y_train, y_pred[:, 1]))
    
for y_pred in ada.staged_predict_proba(X_valid):
    scores_valid.append(roc_auc_score(y_valid, y_pred[:, 1]))

In [ ]:
plt.plot(scores_train, label='train score')
plt.plot(scores_valid, label='valid score')
plt.legend()

In [ ]:
cv = StratifiedKFold(n_splits=4, random_state=123)
scoring = 'roc_auc'

scores = cross_val_score(ada, X, y, scoring=scoring, cv=cv, n_jobs=-1)
scores.mean()

# Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
fig, ax = plt.subplots(1,2)

for lr in [0.02, 0.5, 1]:

    model = GradientBoostingClassifier(n_estimators=300, max_depth=3,
                                       learning_rate=lr, 
                                       random_state=123)
    model.fit(X_train, y_train)

    scores_train = []
    scores_valid = []

    for y_pred in model.staged_predict_proba(X_train):
        scores_train.append(roc_auc_score(y_train, y_pred[:, 1]))
        
    ax[0].plot(scores_train, label='lr={}'.format(lr))
    ax[0].set_title('Train set')

    for y_pred in model.staged_predict_proba(X_valid):
        scores_valid.append(roc_auc_score(y_valid, y_pred[:, 1])) 
        
    ax[1].plot(scores_valid, label='lr={}'.format(lr))
    ax[1].set_title('Validation set')
plt.legend()


In [ ]:
model = GradientBoostingClassifier(n_estimators=300, max_depth=3,
                                       learning_rate=0.02, 
                                       random_state=123)

In [ ]:
scores = cross_val_score(model, X, y, scoring=scoring, cv=cv, n_jobs=-1)
scores.mean()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
imp = pd.Series(index=df_X.columns, data=model.feature_importances_)

# Partial dependency plot

In [ ]:
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence

In [ ]:
fig, ax =\
plot_partial_dependence(model, X_train, [0, 1, 3, (0,3)], 
                        feature_names=dfx.columns)

In [ ]:
pdp, axes = partial_dependence(model, [0], X=X_train, )

In [ ]:
plt.plot(axes[0], pdp[0])

# Other libraries... maybe...